In [250]:
from nautilus_trader.examples.strategies.ema_cross_cython import EMACross

# ENVIRONMENT
import pandas as pd
import dotenv
import os

from nautilus_trader.model import position

dotenv.load_dotenv()
MT5_SERVER = os.environ["MT5_SERVER"]
MT5_LOGIN = os.environ["MT5_LOGIN"]
MT5_PASSWORD = os.environ["MT5_PASSWORD"]
DATA_PATH = os.environ["DATA_PATH"]
CATALOG_PATH = os.path.join(os.getcwd(), os.environ["CATALOG_PATH"])


In [251]:
from nautilus_trader.model.identifiers import Venue, InstrumentId
from nautilus_trader.model.data import Bar, BarType, QuoteTick
from nautilus_trader.config import BacktestVenueConfig, BacktestDataConfig, BacktestRunConfig, BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.trading.strategy import ImportableStrategyConfig
from nautilus_trader.core.datetime import dt_to_unix_nanos, maybe_unix_nanos_to_dt
from nautilus_trader.persistence.catalog import ParquetDataCatalog

from pandas import Timestamp

import data_utils

catalog = ParquetDataCatalog(CATALOG_PATH)

start = dt_to_unix_nanos(pd.Timestamp("2018-08-23 00:00:00"))
end = start + pd.Timedelta(days=5).value 

venue = "SIM_EIGHTCAP"
instrument_id = f"EURUSD.{venue}"


In [252]:
venue_configs = [
    BacktestVenueConfig(
        name=venue,
        oms_type="HEDGING",
        account_type="MARGIN",
        base_currency="USD",
        starting_balances=["10000 USD"],
    ),
]

data_configs = [
    BacktestDataConfig(
        catalog_path=CATALOG_PATH,
        data_cls=QuoteTick,
        instrument_id=instrument_id,
        start_time=start,
        end_time=end,
    ),
]


from decimal import Decimal

strategies = [
    ImportableStrategyConfig(
        strategy_path="nautilus_trader.examples.strategies.ema_cross:EMACross",
        config_path="nautilus_trader.examples.strategies.ema_cross:EMACrossConfig",
        config=dict(
            instrument_id=instrument_id,
            bar_type=f"{instrument_id}-15-MINUTE-MID-INTERNAL",
            fast_ema_period=10,
            slow_ema_period=20,
            trade_size=Decimal(10_000),
        ),
    ),
]

configs = [BacktestRunConfig(
    engine=BacktestEngineConfig(strategies=strategies),
    data=data_configs,
    venues=venue_configs,
)]

node = BacktestNode(configs)

In [253]:
res = node.run()[0]


Failed to set global default dispatcher because of error: a global default trace dispatcher has already been set
2023-12-04T19:49:03.366608001Z [INF] BACKTESTER-001.BacktestEngine: =================================================================
2023-12-04T19:49:03.366628001Z [INF] BACKTESTER-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2023-12-04T19:49:03.366628002Z [INF] BACKTESTER-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2023-12-04T19:49:03.366628003Z [INF] BACKTESTER-001.BacktestEngine:  Copyright (C) 2015-2023. All rights reserved.
2023-12-04T19:49:03.366629001Z [INF] BACKTESTER-001.BacktestEngine: =================================================================
2023-12-04T19:49:03.366629002Z [INF] BACKTESTER-001.BacktestEngine: 
2023-12-04T19:49:03.366629003Z [INF] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2023-12-04T19:49:03.366632001Z [INF] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿⣶⣶⣤⣀⠀⠀⠀⠀⠀
2023-12-04T

In [254]:
res

BacktestResult(trader_id='BACKTESTER-001', machine_id='Tobiass-MacBook-Air.local', run_config_id='820755954084afdb0fefaed40570492dca200931ceacb6bf6b05f71d9ff218f2', instance_id='479d5c27-e91c-48f5-9dea-527d9e0133d7', run_id='3b4d21a8-604d-41f0-a79c-a49946d9db09', run_started=1701719343451034001, run_finished=1701719344698341001, backtest_start=1534982408035000000, backtest_end=1535414396563000000, elapsed_time=431988.528, iterations=0, total_events=40, total_orders=20, total_positions=10, stats_pnls={'USD': {'PnL (total)': 106.2, 'PnL% (total)': 1.0620000000000072, 'Max Winner': 78.04, 'Avg Winner': 39.690000000000005, 'Min Winner': 3.04, 'Min Loser': -4.06, 'Avg Loser': -8.76, 'Max Loser': -13.66, 'Expectancy': 10.620000000000005, 'Win Rate': 0.4}}, stats_returns={'Returns Volatility (252 days)': 0.0963438621809419, 'Average (Return)': 0.0009566834515989467, 'Average Loss (Return)': -0.0007176886942644609, 'Average Win (Return)': 0.003468241670394058, 'Sharpe Ratio (252 days)': 5.0046

In [255]:
from datetime import datetime
print(maybe_unix_nanos_to_dt(res.backtest_start),
maybe_unix_nanos_to_dt(res.backtest_end))

2018-08-23 00:00:08.035000+00:00 2018-08-27 23:59:56.563000+00:00


In [256]:

backtest_start = maybe_unix_nanos_to_dt(res.backtest_start)
backtest_end = maybe_unix_nanos_to_dt(res.backtest_end)

In [257]:
from nautilus_trader.backtest.engine import BacktestResult


In [258]:
engine = node.get_engine(res.run_config_id)

In [259]:
strategy = engine.trader.strategies()[0]

In [260]:
strategy: EMACross = strategy

In [261]:
strategy.registered_indicators

[ExponentialMovingAverage(10), ExponentialMovingAverage(20)]

In [262]:
from nautilus_trader.cache.cache import Cache

cache: Cache = strategy.cache

In [263]:
cache.positions()

[Position(FLAT EURUSD.SIM_EIGHTCAP, id=SIM_EIGHTCAP-1-001),
 Position(FLAT EURUSD.SIM_EIGHTCAP, id=SIM_EIGHTCAP-1-002),
 Position(FLAT EURUSD.SIM_EIGHTCAP, id=SIM_EIGHTCAP-1-003),
 Position(FLAT EURUSD.SIM_EIGHTCAP, id=SIM_EIGHTCAP-1-004),
 Position(FLAT EURUSD.SIM_EIGHTCAP, id=SIM_EIGHTCAP-1-005),
 Position(FLAT EURUSD.SIM_EIGHTCAP, id=SIM_EIGHTCAP-1-006),
 Position(FLAT EURUSD.SIM_EIGHTCAP, id=SIM_EIGHTCAP-1-007),
 Position(FLAT EURUSD.SIM_EIGHTCAP, id=SIM_EIGHTCAP-1-008),
 Position(FLAT EURUSD.SIM_EIGHTCAP, id=SIM_EIGHTCAP-1-009),
 Position(FLAT EURUSD.SIM_EIGHTCAP, id=SIM_EIGHTCAP-1-010)]

In [264]:
cache.orders_closed_count()

20

In [265]:
cache.positions_open()

[]

In [266]:
from nautilus_trader.model.position import Position

pos: Position = (cache.positions_closed())[0]

In [267]:
pos

Position(FLAT EURUSD.SIM_EIGHTCAP, id=SIM_EIGHTCAP-1-001)

In [268]:
pos.realized_pnl

Money('-4.16', USD)

In [269]:
res

BacktestResult(trader_id='BACKTESTER-001', machine_id='Tobiass-MacBook-Air.local', run_config_id='820755954084afdb0fefaed40570492dca200931ceacb6bf6b05f71d9ff218f2', instance_id='479d5c27-e91c-48f5-9dea-527d9e0133d7', run_id='3b4d21a8-604d-41f0-a79c-a49946d9db09', run_started=1701719343451034001, run_finished=1701719344698341001, backtest_start=1534982408035000000, backtest_end=1535414396563000000, elapsed_time=431988.528, iterations=0, total_events=40, total_orders=20, total_positions=10, stats_pnls={'USD': {'PnL (total)': 106.2, 'PnL% (total)': 1.0620000000000072, 'Max Winner': 78.04, 'Avg Winner': 39.690000000000005, 'Min Winner': 3.04, 'Min Loser': -4.06, 'Avg Loser': -8.76, 'Max Loser': -13.66, 'Expectancy': 10.620000000000005, 'Win Rate': 0.4}}, stats_returns={'Returns Volatility (252 days)': 0.0963438621809419, 'Average (Return)': 0.0009566834515989467, 'Average Loss (Return)': -0.0007176886942644609, 'Average Win (Return)': 0.003468241670394058, 'Sharpe Ratio (252 days)': 5.0046

In [270]:
%matplotlib inline

In [278]:
from nautilus_trader.core.rust.model import PositionSide
from nautilus_trader.examples.strategies.ema_cross_cython import EMACrossConfig
import matplotlib.pyplot as plt
import plotly.graph_objects as go

def visualize():
    strategy: EMACross = engine.trader.strategies()[0]
    cache: Cache = strategy.cache
    positions: list[Position] = cache.positions_closed()
    config: EMACrossConfig = strategy.config
    
    bars: list[Bar] = cache.bars(BarType.from_str(config.bar_type))
    
    # bars to df
    df = pd.DataFrame([Bar.to_dict(b)  for b in bars])
    df['ts_event'] = df['ts_event'].apply(maybe_unix_nanos_to_dt)
    
    # plotly candlestick
    # skip weekends because gap in plot 
    # skip on x axis aswell because plotly will plot a gap
    
    # df = df[df['ts_event'].dt.weekday < 5
    
    fig = go.Figure(data=[go.Candlestick(x=df['ts_event'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])
    
    
    fig.update_xaxes(
        rangebreaks=[
            dict(bounds=["sat", "mon"]), # hide weekends
        ]
    )
    
    # add positions to plotly fig without legend
    for pos in positions:
        color = "green" if pos.side == PositionSide.LONG else "red" if pos.side == PositionSide.SHORT else "black"
        fig.add_trace(go.Scatter(
            x=[maybe_unix_nanos_to_dt(pos.ts_opened), maybe_unix_nanos_to_dt(pos.ts_closed)],
            y=[pos.avg_px_open, pos.avg_px_close],
            mode="lines",
            line=dict(color=color, width=2),
            name=f"Position {pos.id}",
            showlegend=False
        ))
    
    fig.show()
    return 
    
    plt.figure(figsize=(10, 8))
    # plot bars close
    plt.plot([maybe_unix_nanos_to_dt(b.ts_event) for b in bars], [b.close for b in bars])  
    # plot bars open
    plt.plot([maybe_unix_nanos_to_dt(b.ts_event) for b in bars], [b.open for b in bars])
    
    # plot positions as line from open to close
    for pos in positions:
        color = "green" if pos.side == PositionSide.LONG else "red" if pos.side == PositionSide.SHORT else "black"
        print(pos.is_long, pos.is_short)
        plt.plot([maybe_unix_nanos_to_dt(pos.ts_opened), maybe_unix_nanos_to_dt(pos.ts_closed)], [pos.avg_px_open, pos.avg_px_close],
                    color=color
                 )
    
    plt.show()

In [279]:
visualize()

In [153]:

cache.positions(side=PositionSide.SHORT)

[]

In [154]:
cache.positions(side=PositionSide.LONG)

[]

In [155]:
cache.positions(side=PositionSide.NO_POSITION_SIDE)

[Position(FLAT EURUSD.SIM_EIGHTCAP, id=SIM_EIGHTCAP-1-001),
 Position(FLAT EURUSD.SIM_EIGHTCAP, id=SIM_EIGHTCAP-1-002),
 Position(FLAT EURUSD.SIM_EIGHTCAP, id=SIM_EIGHTCAP-1-003),
 Position(FLAT EURUSD.SIM_EIGHTCAP, id=SIM_EIGHTCAP-1-004)]